In [47]:
# -*- coding: utf-8 -*-
from selenium import webdriver
import re
import os
import random
# Python이 실행될 때 DJANGO_SETTINGS_MODULE이라는 환경 변수에 현재 프로젝트의 settings.py파일 경로를 등록합니다.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "book.settings")
# 이제 장고를 가져와 장고 프로젝트를 사용할 수 있도록 환경을 만듭니다.
import django
django.setup()
from book.models import Book, MetaData
from django.utils import timezone

'''
    크롬 헤들리스 버전으로 사용할 수 있다고 하는데?
'''
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

'''
    1. 주간 베스트 글을 수집하여 Django ORM으로 DB에 저장.
    2. PageNumber에 1, 2, 3...
'''
max_crawl = 1
urlBase = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001001003&sumgb=06&PageNumber='
bookInfo = ['Year', 'Month', 'Day', 'Rank', 'Title',
    'ISBN', 'Writer', 'Publisher', 'Cate_A', 'Cate_B',
    'Cate_C', 'Cate_D', 'Price', 'Review', 'SellingPoint']

def getDriver():
    try:
        driver = webdriver.Chrome('/Users/canine/DEV/20170114/django_1/BookSeeingEye/book/driver/chromedriver', chrome_options=options)
    except:
        driver = webdriver.Chrome('/Users/canine/DEV/20170114/django_1/BookSeeingEye/book/driver/chromedriver.exe', chrome_options=options)
    return driver

def startCrawl(page, driver):
    if driver == None:
        print('driver is not found')
        return -1
    elif MetaData.objects.filter(crawl_date__range=(start_date, end_date)).count() >= max_crawl:
        print('today crawl is done')
        return -2
    else:
        for pageCount in range(1, page + 1):
            pageCrawl(pageCount, driver)

def endCrawl(driver):
    if driver == None:
        print('driver is not found')
        return -1
    else:
        print('크롤러를 종료합니다.')
        driver.close()
        return 0

def ORM_saveBook(book):
    '''
        book 인스턴스를 받아서 책을 데이터 베이스에 저장함
            - ISBN이 있는 책인 경우에는 MetaData만 저장함
            - ISBN이 없는 책은 책도 새로 등록함
    '''
    if book is not None:
        try:
            temp = Book.objects.get(isbn=int(book['ISBN']))
            MetaData.objects.create(
                rank=book['Rank'],
                reviewCount=book['Review'],
                sellingPoint=book['SellingPoint'],
                crawl_date=timezone.localtime(),
                book=temp
            )
        except:
            MetaData.objects.create(
                rank=book['Rank'],
                reviewCount=book['Review'],
                sellingPoint=book['SellingPoint'],
                crawl_date=timezone.localtime(),
                book=Book.objects.create(
                    title=book['Title'],
                    isbn=int(book['ISBN']),
                    author=book['Writer'],
                    publisher=book['Publisher'],
                    price=book['Price'],
                    pubDate=book['Year'] + '-' + book['Month'] + '-' + book['Day'],
                    category=book['Category'],
                )
            )

def pageCrawl(page, driver):
    if(driver == None):
        print('driver is not found')
        return -1
    else:
        driver.get(urlBase + str(page))

        for i in range(1, 21):
            temp = {}

            pos = str(1 + (i - 1) * 2)
            temp['Rank'] = i + (page - 1) * 20
            item = driver.find_element_by_xpath('//*[@id="category_layout"]/tbody/tr[' + pos + ']/td[3]/p[1]/a[1]')
            temp['Title'] = item.text
            temp['Writer'] = driver.find_element_by_xpath('//*[@id="category_layout"]/tbody/tr[' + pos + ']/td[3]/div/a[1]').text

            driver.execute_script("arguments[0].click();", item)

            try:
                temp['ISBN'] = driver.find_element_by_css_selector('#tblGoodsFairTraderNoti tbody tr:nth-child(3) td').text
            except:
                temp['ISBN'] = 0

            temp['Publisher'] = driver.find_element_by_css_selector('span.gd_pub a').text
            temp['Price'] = int(re.sub('[^0-9]', '', driver.find_element_by_xpath('//*[@id="yDetailTopWrap"]/div[2]/div[2]/div[1]/div[1]/table/tbody/tr[1]/td/span/em').text))

            try:
                temp['Review'] = int(
                    driver.find_element_by_css_selector('span.gd_reviewCount em').text)
            except:
                temp['Review'] = 0

            raw_date = driver.find_element_by_css_selector('span.gd_date').text
            temp['Year'] = raw_date[0:4]
            temp['Month'] = raw_date[6:8]
            temp['Day'] = raw_date[10:12]

            try:
                temp['SellingPoint'] = int(re.sub('[^0-9]', '', driver.find_element_by_css_selector('span.gd_sellNum').text))
            except:
                temp['SellingPoint'] = 'SP_IS_NONE'

            try:       
                temp['Category'] = driver.find_element_by_css_selector('.basicListType ul').text.replace("\n","<br/>")
                print(temp['Category'])
                    
            except:
                temp['Category'] = 'None'

            print(temp)
            ORM_saveBook(temp)
            driver.back()

'''
    날짜 구하기 전용 함수를 모아놓은 곳
        - getTodayEdge: 정확하게 오늘의 끝 시간을 계산
        - getTodayStart: 오늘의 시작 시간을 계산
        ...
'''
def getTodayEdge():
    return getTodayStart() + timezone.timedelta(hours=23, minutes=59, seconds=59)

def getTodayStart():
    return timezone.localtime() - timezone.timedelta(hours=timezone.localtime().hour, minutes=timezone.localtime().minute, seconds=timezone.localtime().second)

def getYesterdayEdge():
    return getTodayStart() - timezone.timedelta(seconds=1)

def getYesterdayStart():
    return getTodayStart() - timezone.timedelta(days=1)

def getBookList(start, end):
    return MetaData.objects.filter(crawl_date__range=(start, end))

'''
    어제 오늘의 rank를 비교하여 Model의 rankRiseAndFall 값에 저장한다.

    알고리즘?
        - 오늘 수집한 책 리스트를 가져온다
        - 어제 수집한 책 리스트를 가져온다
        - 오늘 수집한 책을 기준으로 1~N위까지...
            - isbn이 같으면 순위가 그대로 임
            - isbn이 다르면...
                - 새로운 책이거나
                - 순위가 바뀐 책임
'''
def setRankRiseAndFall(today_list, yesterday_list):
    for item in today_list:
        i = 0
        if(item.book.isbn == yesterday_list.get(book = item.book).book.isbn):
            print(item)
            item.rankRiseAndFall = 0
            print('setRRF is 0')
            item.save()
            # print(today_list.count())
            # print(today_list[i].rankRiseAndFall)
            # print(type(today_list[i].rankRiseAndFall))
            # print('  -  ' + str(today_list[i].rank) + ' ' + str(today_list[i]))
        else:
            try:
                yesterday_list.get(book = item.book)
                riseandfall = yesterday_list.get(book = item.book).rank - item.rank
                item.rankRiseAndFall = str(riseandfall)
                print('setRRF is number')
                item.save()
                # print('  ' + str(riseandfall) + '  ' + str(today_list[i].rank) + str(today_list[i]))
            except MetaData.DoesNotExist:
                item.rankRiseAndFall = str(item.rank) + ' new!'
                print('setRRF is new')
                item.save()
                # print(' new! ' + str(today_list[i].rank) + ' ' +str(today_list[i]))
        
        i += 1
        # today_list[i].save()
        # today_list.filter(rankRiseAndFall)

start_date = getTodayStart()
end_date = getTodayEdge()

In [9]:
temp = getBookList(getTodayStart(), getTodayEdge())

In [6]:
temp

<QuerySet [<MetaData: 윤성우의 열혈 C 프로그래밍>, <MetaData: Do it! 점프 투 파이썬>, <MetaData: 직장인을 위한 실무 엑셀>, <MetaData: 맛있는 디자인 포토샵&일러스트레이터 CC 2017>, <MetaData: 블록체인 무엇인가?>, <MetaData: Do it! 안드로이드 앱 프로그래밍>, <MetaData: 고소한 DIAT 프리젠테이션 파워포인트 2010>, <MetaData: 맛있는 디자인 포토샵 CC 2018>, <MetaData: 두근두근 파이썬>, <MetaData: 밑바닥부터 시작하는 딥러닝>, <MetaData: 된다! 김메주의 유튜브 영상 만들기>, <MetaData: 직장인을 위한 실무 엑셀&파워포인트>, <MetaData: 모두의 파이썬>, <MetaData: 명품 HTML5+CSS3+Javascript 웹 프로그래밍>, <MetaData: C로 배우는 쉬운 자료구조>, <MetaData: NEW 컴퓨터와 인사하기 윈도우 7>, <MetaData: 이것이 자바다>, <MetaData: 4차산업혁명시대 온라인 생존마케팅>, <MetaData: 쉽게 배우는 데이터 통신과 컴퓨터 네트워크>, <MetaData: Do it! 쉽게 배우는 R 데이터 분석>]>

In [7]:
#to Yesterday
for item in temp:
    item.crawl_date = getYesterdayStart() + timezone.timedelta(hours=1, minutes=59, seconds=59)
    item.save()

In [55]:
#delete today
getBookList(getTodayStart(), getTodayEdge()).delete()

(40, {'book.MetaData': 40})

In [40]:
#get rankRiseAndFall
for item in getBookList(getTodayStart(), getTodayEdge()):
    print(item.rankRiseAndFall)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [44]:
#set RRF
setRankRiseAndFall(getBookList(getTodayStart(), getTodayEdge()), getBookList(getYesterdayStart(), getYesterdayEdge()))

윤성우의 열혈 C 프로그래밍
setRRF is 0
Do it! 점프 투 파이썬
setRRF is 0
직장인을 위한 실무 엑셀
setRRF is 0
맛있는 디자인 포토샵&일러스트레이터 CC 2017
setRRF is 0
블록체인 무엇인가?
setRRF is 0
Do it! 안드로이드 앱 프로그래밍
setRRF is 0
고소한 DIAT 프리젠테이션 파워포인트 2010
setRRF is 0
맛있는 디자인 포토샵 CC 2018
setRRF is 0
두근두근 파이썬
setRRF is 0
밑바닥부터 시작하는 딥러닝
setRRF is 0
된다! 김메주의 유튜브 영상 만들기
setRRF is 0
직장인을 위한 실무 엑셀&파워포인트
setRRF is 0
모두의 파이썬
setRRF is 0
명품 HTML5+CSS3+Javascript 웹 프로그래밍
setRRF is 0
C로 배우는 쉬운 자료구조
setRRF is 0
NEW 컴퓨터와 인사하기 윈도우 7
setRRF is 0
이것이 자바다
setRRF is 0
4차산업혁명시대 온라인 생존마케팅
setRRF is 0
쉽게 배우는 데이터 통신과 컴퓨터 네트워크
setRRF is 0
Do it! 쉽게 배우는 R 데이터 분석
setRRF is 0


In [30]:
yes = getBookList(getYesterdayStart(), getYesterdayEdge())

In [31]:
yes

<QuerySet [<MetaData: 윤성우의 열혈 C 프로그래밍>, <MetaData: Do it! 점프 투 파이썬>, <MetaData: 직장인을 위한 실무 엑셀>, <MetaData: 맛있는 디자인 포토샵&일러스트레이터 CC 2017>, <MetaData: 블록체인 무엇인가?>, <MetaData: Do it! 안드로이드 앱 프로그래밍>, <MetaData: 고소한 DIAT 프리젠테이션 파워포인트 2010>, <MetaData: 맛있는 디자인 포토샵 CC 2018>, <MetaData: 두근두근 파이썬>, <MetaData: 밑바닥부터 시작하는 딥러닝>, <MetaData: 된다! 김메주의 유튜브 영상 만들기>, <MetaData: 직장인을 위한 실무 엑셀&파워포인트>, <MetaData: 모두의 파이썬>, <MetaData: 명품 HTML5+CSS3+Javascript 웹 프로그래밍>, <MetaData: C로 배우는 쉬운 자료구조>, <MetaData: NEW 컴퓨터와 인사하기 윈도우 7>, <MetaData: 이것이 자바다>, <MetaData: 4차산업혁명시대 온라인 생존마케팅>, <MetaData: 쉽게 배우는 데이터 통신과 컴퓨터 네트워크>, <MetaData: Do it! 쉽게 배우는 R 데이터 분석>]>

In [51]:
# set yesterday book random
yes = getBookList(getYesterdayStart(), getYesterdayEdge())
for item in yes:
    item.rank = random.randrange(1, 100)
    item.save()
    
for item in yes:
    print(str(item.rank) + " " + str(item.book.title))

90 윤성우의 열혈 C 프로그래밍
7 Do it! 점프 투 파이썬
96 직장인을 위한 실무 엑셀
84 맛있는 디자인 포토샵&일러스트레이터 CC 2017
4 블록체인 무엇인가?
18 Do it! 안드로이드 앱 프로그래밍
26 고소한 DIAT 프리젠테이션 파워포인트 2010
24 맛있는 디자인 포토샵 CC 2018
56 두근두근 파이썬
40 밑바닥부터 시작하는 딥러닝
70 된다! 김메주의 유튜브 영상 만들기
49 직장인을 위한 실무 엑셀&파워포인트
66 모두의 파이썬
11 명품 HTML5+CSS3+Javascript 웹 프로그래밍
30 C로 배우는 쉬운 자료구조
50 NEW 컴퓨터와 인사하기 윈도우 7
22 이것이 자바다
48 4차산업혁명시대 온라인 생존마케팅
95 쉽게 배우는 데이터 통신과 컴퓨터 네트워크
91 Do it! 쉽게 배우는 R 데이터 분석


In [57]:
# category
today = getBookList(getTodayStart(), getTodayEdge())
for item in today:
    print(item.book.category)

국내도서 > IT 모바일 > 프로그래밍 언어 > C
국내도서 > IT 모바일 > 프로그래밍 언어 > 파이썬
국내도서 > IT 모바일 > 오피스 활용 > MS Excel(엑셀)
국내도서 > IT 모바일 > 그래픽/디자인/멀티미디어 > 일러스트레이터<br/>국내도서 > IT 모바일 > 그래픽/디자인/멀티미디어 > 포토샵
국내도서 > IT 모바일 > 네트워크/해킹/보안 > 보안/해킹<br/>국내도서 > 경제 경영 > 마케팅/세일즈 > 트렌드/미래예측<br/>국내도서 > 경제 경영 > 경제 > 각국 경제/경제사/전망 > 세계경제
국내도서 > IT 모바일 > 모바일 프로그래밍 > 안드로이드폰
국내도서 > IT 모바일 > 오피스 활용 > 프레젠테이션 > 파워포인트
국내도서 > IT 모바일 > 그래픽/디자인/멀티미디어 > 포토샵
국내도서 > IT 모바일 > 프로그래밍 언어 > 파이썬
국내도서 > IT 모바일 > 프로그래밍 언어 > 파이썬<br/>국내도서 > IT 모바일 > OS/데이터베이스 > 클라우드/빅데이터<br/>국내도서 > IT 모바일 > 컴퓨터 공학 > 자료구조/알고리즘<br/>국내도서 > IT 모바일 > 컴퓨터 공학 > 인공지능<br/>국내도서 > 네티즌 선정 올해의 책 > 2017년 올해의 책 후보도서
국내도서 > IT 모바일 > 웹사이트 > 블로그/홈페이지 만들기<br/>국내도서 > IT 모바일 > 모바일/태블릿/SNS > 유투브<br/>국내도서 > IT 모바일 > 인터넷 비즈니스 > 인터넷 마케팅
국내도서 > IT 모바일 > 오피스 활용 > MS Excel(엑셀)<br/>국내도서 > IT 모바일 > 오피스 활용 > 프레젠테이션 > 파워포인트
국내도서 > IT 모바일 > 컴퓨터 입문/활용 > 어린이 컴퓨터<br/>국내도서 > IT 모바일 > 프로그래밍 언어 > 파이썬<br/>국내도서 > 어린이 > 초등학습 > 컴퓨터/IT/코딩<br/>국내도서 > 어린이 > 5-6학년 > 5-6학년 학습 > 5-6학년 컴퓨터
국내도서 > IT 모바일 > 웹사이트 > HTML/